In [1]:
import open_clip 
from PIL import Image 
from arguments import parser 
import torch 
import torch.nn as nn 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn.manifold import TSNE
import os 
from datasets import create_dataset 
from torch.utils.data import DataLoader
from utils.metrics import MetricCalculator, loco_auroc
from accelerate import Accelerator
from omegaconf import OmegaConf
import seaborn as sns 
from models import LANGCAD 

os.environ['CUDA_VISIBLE_DEVICES'] = '0' 
default_setting = './configs/default/visa.yaml'
model_setting = './configs/model/LANGCAD.yaml'
cfg = parser(True,default_setting, model_setting)


model  = __import__('models').__dict__[cfg.MODEL.method](
        backbone = cfg.MODEL.backbone,
        **cfg.MODEL.params
        )
device = 'cuda'


loader_dict = {}
accelerator = Accelerator()
for cn in cfg.DATASET.class_names:
    trainset, testset = create_dataset(
        dataset_name  = cfg.DATASET.dataset_name,
        datadir       = cfg.DATASET.datadir,
        class_name    = cn,
        img_size      = cfg.DATASET.img_size,
        mean          = cfg.DATASET.mean,
        std           = cfg.DATASET.std,
        aug_info      = cfg.DATASET.aug_info,
        **cfg.DATASET.get('params',{})
    )
    trainloader = DataLoader(
        dataset     = trainset,
        batch_size  = cfg.DATASET.batch_size,
        num_workers = cfg.DATASET.num_workers,
        shuffle     = True 
    )    

    testloader = DataLoader(
            dataset     = testset,
            batch_size  = cfg.DATASET.batch_size,
            num_workers = cfg.DATASET.num_workers,
            shuffle     = False 
        )    
    
    loader_dict[cn] = {'train':trainloader,'test':testloader}
    
    pool = [] 
    model, trainloader, testloader = accelerator.prepare(model, trainloader, testloader)
    for imgs, img_dirs in trainloader:
        features = model.embed(imgs)
        pool.append(
            features.detach().cpu().numpy()
        )
    pool = np.concatenate(pool)   
    knowledge = model.fit(pool)
    model.pool.get_knowledge(cn, knowledge)
    break 


 Experiment Name : CViT_grid_first



Subsampling...: 100%|██████████| 17718/17718 [00:04<00:00, 3864.66it/s]

Memory bank fit


In [3]:
from utils.metrics import MetricCalculator

model.eval()
img_level = MetricCalculator(metric_list = ['auroc','average_precision'])
pix_level = MetricCalculator(metric_list = ['auroc','average_precision'])    


for imgs, labels, gts in testloader:
    s, score, score_map  = model.predict(imgs)
    pix_level.update(score_map,gts.type(torch.int))
    img_level.update(score, labels.type(torch.int))

p_results = pix_level.compute()
i_results = img_level.compute()
    